<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Wikipedia-page-scaping-using-urllib-and-beautifulsoup" data-toc-modified-id="Wikipedia-page-scaping-using-urllib-and-beautifulsoup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Wikipedia page scaping using urllib and beautifulsoup</a></span></li><li><span><a href="#Load-scraped-dataset-to-pandas-dataframe" data-toc-modified-id="Load-scraped-dataset-to-pandas-dataframe-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load scraped dataset to pandas dataframe</a></span></li><li><span><a href="#Data-cleaning" data-toc-modified-id="Data-cleaning-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data cleaning</a></span></li><li><span><a href="#Retrieve-Geolocations-from-geocoder" data-toc-modified-id="Retrieve-Geolocations-from-geocoder-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Retrieve Geolocations from geocoder</a></span></li><li><span><a href="#Create-map-of-the-data" data-toc-modified-id="Create-map-of-the-data-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Create map of the data</a></span></li><li><span><a href="#Define-Foursquare-credentials-and-version" data-toc-modified-id="Define-Foursquare-credentials-and-version-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Define Foursquare credentials and version</a></span><ul class="toc-item"><li><span><a href="#Extract-boroughs-that-contain-Toronto" data-toc-modified-id="Extract-boroughs-that-contain-Toronto-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Extract boroughs that contain Toronto</a></span></li></ul></li><li><span><a href="#Clustering-Neighborhoods" data-toc-modified-id="Clustering-Neighborhoods-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Clustering Neighborhoods</a></span><ul class="toc-item"><li><span><a href="#One-hot-encoding-for-category-features" data-toc-modified-id="One-hot-encoding-for-category-features-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>One-hot encoding for category features</a></span></li><li><span><a href="#K-Means" data-toc-modified-id="K-Means-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>K-Means</a></span></li></ul></li></ul></div>

# Wikipedia page scaping using urllib and beautifulsoup

In [60]:
# import the library to open URLs
import urllib.request

# import the BeautifulSoup library to parse HTML and XML documents
from bs4 import BeautifulSoup

import pandas as pd

In [61]:
# specify which URL/web page we are going to be scraping
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [62]:
# open the url using urllib.request and put the HTML into the page variable
page = urllib.request.urlopen(url)

In [63]:
# parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page, "lxml")

In [64]:
#TO check the html underlying the chosen webpage
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XsdgLApAIHkAAEZUQdYAAAAS","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":958152029,"wgRevisionId":958152029,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toronto","Ontario

In [65]:
#Retrieve tables from the webpage
table = soup.find('table', class_ = 'wikitable sortable')
#table

In [66]:
A = []
B = []
C = []

for row in table.find_all('tr'):
    cells = row.find_all('td')
    if len(cells) == 3:
        A.append(cells[0].find(text = True).strip())
        B.append(cells[1].find(text = True).strip())
        C.append(cells[2].find(text = True).strip())

# Load scraped dataset to pandas dataframe

In [67]:
df = pd.DataFrame(columns = ['PostalCode', 'Borough', 'Neighborhood'])
df['PostalCode'] = A
df['Borough'] = B
df['Neighborhood'] = C

In [68]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


# Data cleaning

In [69]:
# Only process the cells that have an assigned borough
df_cleaned = df.copy()
df_cleaned['Borough'] = df_cleaned['Borough'].astype(str)
df_cleaned = df[(df['Borough'] != 'Not assigned')].reset_index()


In [70]:
df_cleaned.head()

,index,PostalCode,Borough,Neighborhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [71]:
#Check if empty neighbourhood

df_empty = df_cleaned.copy()
df_empty['empty_list'] = np.where(len(df_empty['Neighborhood']) == 0, 'empty','Not empty' )

df_empty[df_empty['empty_list'] == 'empty']

,index,PostalCode,Borough,Neighborhood,empty_list


In [104]:
#Check if 1 postal code has more than 1 neighborhood rows
a = df_cleaned.groupby(['PostalCode','Borough'], as_index = False).agg({'Neighborhood': 'count'})
a[a['Neighborhood'] >1]

,PostalCode,Borough,Neighborhood


In [105]:
df_cleaned.head()

,index,PostalCode,Borough,Neighborhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [99]:
# Shape of the dataframe

df_cleaned.shape

(103, 4)

# Retrieve Geolocations from geocoder 

In [158]:
import requests

#Removed API key
GOOGLE_API_KEY = None 

def extract_lat_long_via_address(postal_code):
    print(postal_code)
    lat, lng = None, None
    api_key = GOOGLE_API_KEY
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    endpoint = f"{base_url}?address={'{}, Toronto, Ontario'.format(postal_code)}&key={api_key}"
    # see how our endpoint includes our API key? Yes this is yet another reason to restrict the key
    r = requests.get(endpoint)
    print(requests)
    if r.status_code not in range(200, 299):
        return None, None
    try:
        '''
        This try block incase any of our inputs are invalid. This is done instead
        of actually writing out handlers for all kinds of responses.
        '''
        results = r.json()['results'][0]
        lat = results['geometry']['location']['lat']
        lng = results['geometry']['location']['lng']
    except:
        pass
    print(lat,lng)
    return lat, lng

df_cleaned['Latitude'], df_cleaned['Longitude'] = zip(*df_cleaned.apply(lambda x: extract_lat_long_via_address(str(x['PostalCode'])), axis = 1))
    

M3A
<module 'requests' from 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\requests\\__init__.py'>
43.7532586 -79.3296565
M4A
<module 'requests' from 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\requests\\__init__.py'>
43.72588229999999 -79.3155716
M5A
<module 'requests' from 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\requests\\__init__.py'>
43.6542599 -79.36063589999999
M6A
<module 'requests' from 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\requests\\__init__.py'>
43.718518 -79.4647633
M7A
<module 'requests' from 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\requests\\__init__.py'>
None None
M9A
<module 'requests' from 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\requests\\__init__.py'>
43.6678556 -79.5322424
M1B
<module 'requests' from 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\requests\\__init__.py'>
43.8066863 -79.1943534
M3B
<module 'requests' from 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\requests\\__init__.py'>
43.7459058 -79.352188
M4B
<modu

<module 'requests' from 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\requests\\__init__.py'>
43.7127511 -79.3901975
M5P
<module 'requests' from 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\requests\\__init__.py'>
43.6969476 -79.4113072
M6P
<module 'requests' from 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\requests\\__init__.py'>
43.6616083 -79.4647633
M9P
<module 'requests' from 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\requests\\__init__.py'>
43.696319 -79.5322424
M1R
<module 'requests' from 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\requests\\__init__.py'>
43.7500715 -79.2958491
M2R
<module 'requests' from 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\requests\\__init__.py'>
43.7827364 -79.4422593
M4R
<module 'requests' from 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\requests\\__init__.py'>
43.7153834 -79.4056784
M5R
<module 'requests' from 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\requests\\__init__.py'>
43.6727097 -79.4056784
M6R
<module '

In [162]:
# Replace null values in geolocation from reference file

geofile = pd.read_csv('Geospatial_Coordinates.csv')
geofile = geofile.rename(columns = {'Postal Code': 'PostalCode','Latitude': 'Refer_Latitude', 'Longitude': 'Refer_Longitude'})

df_cleaned = pd.merge(df_cleaned, geofile, how = 'left', on = ['PostalCode'])

In [166]:
df_cleaned['Latitude'] = np.where(df_cleaned['Latitude'].isna(), df_cleaned['Refer_Latitude'], df_cleaned['Latitude'] )
df_cleaned['Longitude'] = np.where(df_cleaned['Longitude'].isna(), df_cleaned['Refer_Longitude'], df_cleaned['Longitude'] )

In [170]:
neighborhoods = df_cleaned[['PostalCode', 'Borough', 'Neighborhood', 'Latitude','Longitude']]

In [171]:
neighborhoods.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7185,-79.4648
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895


In [172]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


# Create map of the data

In [173]:
import folium # map rendering library

# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

NameError: name 'folium' is not defined

# Define Foursquare credentials and version

In [180]:
CLIENT_ID = 'ISYKRQQQFB5ZSRCK0XYKXKTG5JO4RYVSBJDJZSHCHLENCJ12' # your Foursquare ID
CLIENT_SECRET = 'OLVPUND5SRFGJW4XFM53OIBCCUALZ5YIHBJK0XROKA4P5M2Q' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 30
radius = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ISYKRQQQFB5ZSRCK0XYKXKTG5JO4RYVSBJDJZSHCHLENCJ12
CLIENT_SECRET:OLVPUND5SRFGJW4XFM53OIBCCUALZ5YIHBJK0XROKA4P5M2Q


## Extract boroughs that contain Toronto 

In [181]:
df_toronto = neighborhoods[neighborhoods['Borough'].str.contains('Toronto')]

In [182]:
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3789
15,M5C,Downtown Toronto,St. James Town,43.6515,-79.3754
19,M4E,East Toronto,The Beaches,43.6764,-79.293


In [183]:
# TO retrieve popular spots around each neighborhood
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [184]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town,

In [185]:
print(toronto_venues.shape)
toronto_venues.head()

(865, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [186]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,30,30,30,30,30,30
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
Business reply mail Processing Centre,17,17,17,17,17,17
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,30,30,30,30,30,30
Christie,17,17,17,17,17,17
Church and Wellesley,30,30,30,30,30,30
"Commerce Court, Victoria Hotel",30,30,30,30,30,30
Davisville,30,30,30,30,30,30


In [188]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 194 uniques categories.


# Clustering Neighborhoods

## One-hot encoding for category features

In [189]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [190]:
toronto_onehot.shape

(865, 194)

In [191]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Business reply mail Processing Centre,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.066667,0.066667,0.066667,0.066667,0.2,0.133333,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.033333,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.033333,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [192]:
toronto_grouped.shape

(39, 194)

In [193]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0  Seafood Restaurant  0.07
1        Cocktail Bar  0.07
2         Coffee Shop  0.07
3            Beer Bar  0.07
4              Bakery  0.03


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.13
1     Coffee Shop  0.09
2  Breakfast Spot  0.09
3       Nightclub  0.09
4    Intersection  0.04


----Business reply mail Processing Centre----
                  venue  freq
0  Gym / Fitness Center  0.06
1                Garden  0.06
2            Comic Shop  0.06
3                  Park  0.06
4      Recording Studio  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0      Airport Service  0.20
1     Airport Terminal  0.13
2  Rental Car Location  0.07
3             Boutique  0.07
4     Sculpture Garden  0.07


----Central Bay Street----
             venue  freq
0      Coffee Shop  0.23
1             C

In [195]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [196]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Seafood Restaurant,Bistro,Jazz Club,Farmers Market,Basketball Stadium,Comfort Food Restaurant,Breakfast Spot
1,"Brockton, Parkdale Village, Exhibition Place",Café,Nightclub,Coffee Shop,Breakfast Spot,Bakery,Performing Arts Venue,Convenience Store,Climbing Gym,Restaurant,Burrito Place
2,Business reply mail Processing Centre,Auto Workshop,Butcher,Light Rail Station,Skate Park,Smoke Shop,Restaurant,Burrito Place,Recording Studio,Farmers Market,Fast Food Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Boat or Ferry,Plane,Airport,Airport Food Court,Airport Gate,Airport Lounge,Boutique,Rental Car Location
4,Central Bay Street,Coffee Shop,Café,Yoga Studio,Spa,Seafood Restaurant,Sandwich Place,Bubble Tea Shop,Ramen Restaurant,Poke Place,Modern European Restaurant


## K-Means 

In [199]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 0, 1, 1, 1, 1, 0])

In [202]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606,0,Coffee Shop,Park,Breakfast Spot,Theater,Bakery,Pub,Distribution Center,Restaurant,Dessert Shop,Event Space
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895,0,Coffee Shop,Sushi Restaurant,Wings Joint,Distribution Center,Park,Mexican Restaurant,Japanese Restaurant,Italian Restaurant,Hobby Shop,Gym
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3789,1,Café,Coffee Shop,Theater,Middle Eastern Restaurant,Diner,Ramen Restaurant,Sandwich Place,Shopping Mall,Electronics Store,Hotel
15,M5C,Downtown Toronto,St. James Town,43.6515,-79.3754,1,Gastropub,Café,Cocktail Bar,Coffee Shop,Vegetarian / Vegan Restaurant,Gym,Park,Middle Eastern Restaurant,Cosmetics Shop,Creperie
19,M4E,East Toronto,The Beaches,43.6764,-79.293,1,Trail,Health Food Store,Pub,Wings Joint,Dance Studio,Electronics Store,Eastern European Restaurant,Donut Shop,Dog Run,Distribution Center


In [204]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

NameError: name 'folium' is not defined